<a href="https://colab.research.google.com/github/amankiitg/Foundation_AI/blob/main/Diffusion_Project_Generate_Egyptian_Characters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Links to Download

Han characters dataset: https://drive.google.com/file/d/1JFU5QRn3T11adAYQtA-Of6-rplfVLyOm/view?usp=sharing

NotoSans Font: https://drive.google.com/file/d/16Qv5S6_ZV3DzLkpvU-MEbCq94m5Olw1F/view?usp=sharing

### Step 1: Generate a Chinese glyph and definition dataset
We use the unicode consortium's [unihan database](https://github.com/unicode-org/unihan-database) for this. We can take the kDefinition text file to extract both characters and labels in the form of definitions.

In [1]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 44.7 MB/s eta 0:00:00


In [2]:
# import fitz  # PyMuPDF
# import re

# # Load your PDF
# file_path = "egyptian.pdf"  # <-- replace with your actual file path
# doc = fitz.open(file_path)

# # Extract text from pages 17 to 83 (index 16 to 82)
# ocr_text = ""
# for page_num in range(16, 83):
#     page = doc.load_page(page_num)
#     ocr_text += page.get_text()

# # Regex pattern for extracting code + glyph + name
# pattern = re.compile(r"(\b1[3-4][0-9A-F]{3,4})\s+([^\s])\s+([A-Z \-0-9]+.*?)\s*(?=\n|$)")
# matches = pattern.findall(ocr_text)

# # Format matches
# formatted_entries = [f"{code} {glyph} {name.strip()}" for code, glyph, name in matches]

# # Save to a .txt file
# output_path = "egyptian_hieroglyphs_extracted.txt"
# with open(output_path, "w", encoding="utf-8") as f:
#     for entry in formatted_entries:
#         f.write(entry + "\n")

# print(f"Saved {len(formatted_entries)} entries to {output_path}")


In [3]:
import fitz  # PyMuPDF
from PIL import Image
import os
import numpy as np
import cv2

def extract_region_from_pdf(pdf_path, page_number, coords, dpi=300, save_path='output.png'):
    """
    Extract a region from a PDF page as a high-quality image.

    Args:
        pdf_path (str): Path to the PDF file.
        page_number (int): Page number (0-indexed).
        coords (tuple): (x0, y0, x1, y1) in points (1/72 inch).
        dpi (int): Resolution for rendering the PDF.
        save_path (str): Where to save the extracted image.

    Returns:
        PIL.Image.Image: Extracted image region.
    """
    output_dir = 'png_matrix'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")

    # Open the PDF
    doc = fitz.open(pdf_path)
    page = doc.load_page(page_number)

    # Calculate zoom factor from DPI
    zoom = dpi / 72
    mat = fitz.Matrix(zoom, zoom)

    # Render the page to a high-res image
    pix = page.get_pixmap(matrix=mat, alpha=False)

    # Convert to PIL Image
    image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    # Scale coordinates for high DPI
    x0, y0, x1, y1 = [int(c * zoom) for c in coords]

    # Crop and save
    cropped = image.crop((x0, y0, x1, y1))
    cropped.save(output_dir+'/'+str(page_number)+save_path)

    return cropped

region_coords = (68, 93, 512, 726)  # x0, y0, x1, y1 in PDF points (1/72")
for p in range(2,16,2):
  extract_region_from_pdf('egyptian.pdf', p, region_coords, dpi=300, save_path='cropped_image.png')

region_coords = (101, 93, 544, 726)  # x0, y0, x1, y1 in PDF points (1/72")
for p in [1]:
  extract_region_from_pdf('egyptian.pdf', p, region_coords, dpi=300, save_path='cropped_image.png')

region_coords = (100, 93, 544, 726)  # x0, y0, x1, y1 in PDF points (1/72")
for p in range(3,16,2):
  extract_region_from_pdf('egyptian.pdf', p, region_coords, dpi=300, save_path='cropped_image.png')

Created output directory: png_matrix


In [4]:
import os
import pandas as pd
import numpy as np
from PIL import Image

def extract_grid_images(df, i, input_image_path, output_folder, grid_size=(16, 16), trim_px=(1, 1, 1, 0)):
    """
    Extract images from a grid and save them as separate files with boundary trimming.

    Args:
        input_image_path (str): Path to the input image containing the grid
        output_folder (str): Folder where extracted images will be saved
        grid_size (tuple): Number of rows and columns in the grid (rows, cols)
        trim_px (tuple): Pixels to trim from each side (left, top, bottom, right)
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output directory: {output_folder}")

    # Load the image
    print(f"Loading image from: {input_image_path}")
    img = Image.open(input_image_path)

    # Get image dimensions
    img_width, img_height = img.size

    # Calculate the size of each cell
    cell_width = img_width / grid_size[1]
    cell_height = img_height / grid_size[0]

    # print(f"Image dimensions: {img_width}x{img_height}")
    # print(f"Cell dimensions: {cell_width}x{cell_height}")

    # Extract and save each cell
    total_cells = grid_size[0] * grid_size[1]
    saved_count = 0

    for row in range(grid_size[0]):
        for col in range(grid_size[1]):
            # Calculate coordinates with boundary trimming
            left = col * cell_width + trim_px[0]
            upper = row * cell_height + trim_px[1]
            right = (col + 1) * cell_width - trim_px[3]
            lower = (row + 1) * cell_height - trim_px[2]

            # print((left, upper, right, lower))
            # Crop the cell
            cell = img.crop((left, upper, right, lower))

            # Generate filename with row and column info
            # Using hex codes from the image (13D60 + row*16 + col)
            hex_code = format(0x13D60 + row * 16 + col, 'X')
            filename = str(i)+str(hex_code)+".png"
            df.loc[len(df.index)] = [str(i)+str(hex_code), filename]
            filepath = os.path.join(output_folder, filename)

            # Save the cell
            cell = cell.resize((128, 128), Image.Resampling.LANCZOS)
            cell.save(filepath)
            saved_count += 1

            # Print progress
            # if saved_count % 16 == 0:
            #     print(f"Progress: {saved_count}/{total_cells} images saved")

    print(f"Extraction complete! {saved_count} images saved to {output_folder}")

df = pd.DataFrame(columns=['Unicode','filename'])
# for i in range(1,2):
#   input_image = "png_matrix/"+str(i)+"cropped_image.png"  # Path to your grid image
#   output_folder = "data"#+str(i)  # Output folder name
#   extract_grid_images(df, i, input_image, output_folder, trim_px=(3, 3, 30, 3))

# Full usage
for i in range(1,11):
  input_image = "png_matrix/"+str(i)+"cropped_image.png"  # Path to your grid image
  output_folder = "data"#+str(i)  # Output folder name
  extract_grid_images(df, i, input_image, output_folder, trim_px=(3, 3, 30, 3))

for i in range(11,16):
  input_image = "png_matrix/"+str(i)+"cropped_image.png"  # Path to your grid image
  output_folder = "data"#+str(i)  # Output folder name
  extract_grid_images(df, i, input_image, output_folder, grid_size=(16, 15), trim_px=(3, 3, 30, 3))

Created output directory: data
Loading image from: png_matrix/1cropped_image.png
Extraction complete! 256 images saved to data
Loading image from: png_matrix/2cropped_image.png
Extraction complete! 256 images saved to data
Loading image from: png_matrix/3cropped_image.png
Extraction complete! 256 images saved to data
Loading image from: png_matrix/4cropped_image.png
Extraction complete! 256 images saved to data
Loading image from: png_matrix/5cropped_image.png
Extraction complete! 256 images saved to data
Loading image from: png_matrix/6cropped_image.png
Extraction complete! 256 images saved to data
Loading image from: png_matrix/7cropped_image.png
Extraction complete! 256 images saved to data
Loading image from: png_matrix/8cropped_image.png
Extraction complete! 256 images saved to data
Loading image from: png_matrix/9cropped_image.png
Extraction complete! 256 images saved to data
Loading image from: png_matrix/10cropped_image.png
Extraction complete! 256 images saved to data
Loading 

In [5]:
df.head()

,Unicode,filename
0,113D60,113D60.png
1,113D61,113D61.png
2,113D62,113D62.png
3,113D63,113D63.png
4,113D64,113D64.png


### Step 2: Create the dataset of images

(1) Take a DataFrame df (like the one you previously created from Unihan-kDefinition.txt)

(2) Extract Unicode characters and render them as images using the specified font style (e.g., 128x128 PNGs).

(3) Save them into a directory using their Unicode code points as filenames.

Generate metadata.jsonl

In [6]:
# # Convert to JSON Lines format
# jsonl_string_with_splits = df.to_json(orient='records', lines=True)

# # Write to metadata.jsonl file
# with open('metadata.jsonl', 'w') as file:
#     file.write(jsonl_string_with_splits)

### Step 3: Converts the DataFrame to JSON Lines format (.jsonl)

Each line is a separate JSON object, which is ideal for training datasets in ML frameworks like Hugging Face, PyTorch, etc.

### Step 4: Define training model configuration

In [7]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    image_size = 128  # assumes images are square
    train_batch_size = 32
    eval_batch_size = 32
    num_epochs = 30
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 1
    save_model_epochs = 30
    mixed_precision = "fp16"  # `no` for float32, `fp16` for automatic mixed precision
    output_dir = "glyffuser-unconditional"  # the model name
    overwrite_output_dir = True  # overwrite the old model when re-running the notebook
    seed = 0
    dataset_name="data128"

config = TrainingConfig()

### Step 5: Create Local Dataset Class and helper functions

In [8]:
!pip install datasets
from datasets import Dataset
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import torchvision.transforms as T
import torch.nn.functional as F
from diffusers import UNet2DModel, DDPMScheduler, DPMSolverMultistepScheduler
from diffusers.optimization import get_cosine_schedule_with_warmup
from diffusers import DDPMPipeline
from accelerate import Accelerator
import torch
from tqdm.auto import tqdm
import os

def normalize_neg_one_to_one(img):
    return img * 2 - 1

# UPDATED LocalDataset class
class LocalDataset(Dataset):
    def __init__(self, folder, image_size, exts=['png']):
        super().__init__()
        self.folder = folder
        self.image_size = image_size

        # Match extensions case-insensitively
        self.paths = []
        for ext in exts:
            self.paths.extend([
                p for p in Path(folder).rglob(f'*.{ext}')
                if p.suffix.lower() == f'.{ext.lower()}'
            ])

        assert len(self.paths) > 0, f"No images found in {folder}. Check path and extensions."

        self.transform = T.Compose([
            T.Resize((image_size, image_size)),
            T.ToTensor(),
            T.Lambda(normalize_neg_one_to_one),
        ])

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        path = self.paths[index]
        img = Image.open(path).convert('L')  # Grayscale
        return self.transform(img)

def make_grid(images, rows, cols):
    # Helper function for making a grid of images
    w, h = images[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    for i, image in enumerate(images):
        grid.paste(image, box=(i%cols*w, i//cols*h))
    return grid

def evaluate(config, epoch, pipeline):
    # Sample from the model and save the images in a grid
    images = pipeline(
        batch_size=config.eval_batch_size,
        generator=torch.Generator(device='cpu').manual_seed(config.seed),
        num_inference_steps=50
    ).images

    image_grid = make_grid(images, rows=4, cols=4)

    test_dir = os.path.join(config.output_dir, "samples")
    os.makedirs(test_dir, exist_ok=True)
    image_grid.save(f"{test_dir}/{epoch:04d}.png")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

### Step 6: Define model training loop

In [9]:
def train_loop(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="tensorboard",
        project_dir=os.path.join(config.output_dir, "logs")
    )

    if accelerator.is_main_process:
        if config.output_dir is not None:
            os.makedirs(config.output_dir, exist_ok=True)
        accelerator.init_trackers("train_example")

    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, lr_scheduler
    )

    global_step = 0

    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):
            clean_images = batch
            noise = torch.randn(clean_images.shape).to(clean_images.device)
            bs = clean_images.shape[0]
            timesteps = torch.randint(0, noise_scheduler.num_train_timesteps, (bs,), device=clean_images.device).long()
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            with accelerator.accumulate(model):
                noise_pred = model(noisy_images, timesteps, return_dict=False)[0]
                loss = F.mse_loss(noise_pred, noise)
                accelerator.backward(loss)
                accelerator.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1

        if accelerator.is_main_process:
            if (epoch + 1) % config.save_image_epochs == 0 or epoch == config.num_epochs - 1:
                pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=inference_scheduler)
                evaluate(config, epoch, pipeline)

            if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
                pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=inference_scheduler)
                save_dir = os.path.join(config.output_dir, f"epoch{epoch}")
                pipeline.save_pretrained(save_dir)

### Step 7: Load dataset

In [10]:
# Define data source
dataset = LocalDataset("data", image_size=config.image_size)
train_dataloader = DataLoader(dataset, batch_size=config.train_batch_size, shuffle=True)

### Step 8: Define UNet Model

In [11]:
# Define model
model = UNet2DModel(
    sample_size=config.image_size,  # the target image resolution
    in_channels=1,  # the number of input channels
    out_channels=1,  # the number of output channels
    layers_per_block=1,  # how many ResNet layers to use per UNet block
    block_out_channels=(128, 128, 256, 256, 512, 512),  # the number of output channels for each UNet block
    down_block_types=(
        "DownBlock2D",
        "DownBlock2D",
        "DownBlock2D",
        "DownBlock2D",
        "AttnDownBlock2D",
        "DownBlock2D",
    ),
    up_block_types=(
        "UpBlock2D",
        "AttnUpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
    ),
)

### Step 9: Define optimizers and schedulers

In [12]:
noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
inference_scheduler = DPMSolverMultistepScheduler()
optimizer = AdamW(model.parameters(), lr=config.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(train_dataloader) * config.num_epochs),
)

### Step 10: Run training on the model!

In [13]:
import pandas as pd
import os
from PIL import Image, ImageDraw, ImageFont

In [ ]:
from accelerate import notebook_launcher

args = (config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler)
notebook_launcher(train_loop, args, num_processes=1)

Launching training on one GPU.


  0%|          | 0/118 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/diffusers/configuration_utils.py:140: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDPMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDPMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


### Step 11: Sample from trained diffusion model

Loads a pretrained DDPMPipeline from epoch99 of a local model directory (likely trained on glyphs or characters).

Moves the model to GPU.

Sets the scheduler to DPMSolverMultistepScheduler, which controls the denoising steps during sampling.

Generates 16 images in one batch.

Uses a fixed random seed (config.seed) for reproducibility.

Uses 50 diffusion steps to sample clean images from noise.

Arranges the 16 images into a 4x4 grid.

Creates a samples folder inside your config.output_dir.

Saves the resulting image grid as samples.png.

In [ ]:
model_path = "glyffuser-unconditional/epoch29"  # Path to the specific epoch model directory# Path to the model directory
pipeline = DDPMPipeline.from_pretrained(model_path).to("cuda")
pipeline.scheduler = DPMSolverMultistepScheduler()

# Sample from the model and save the images in a grid
images = pipeline(
    batch_size=16,
    generator=torch.Generator(device='cuda').manual_seed(config.seed), # Generator can be on GPU here
    num_inference_steps=50
).images

# Make a grid out of the inverted images
image_grid = make_grid(images, rows=4, cols=4)

# Save the images
test_dir = os.path.join(config.output_dir, "samples")
os.makedirs(test_dir, exist_ok=True)
image_grid.save(f"{test_dir}/samples.png")

### Step 12: Visualize model generated images

In [ ]:
from diffusers import DPMSolverMultistepScheduler, DDPMPipeline
from PIL import Image, ImageDraw, ImageFont
import torch

def make_labeled_grid(images, prompt, steps, font_path=None, font_size=20, margin=10):
    assert len(images) == len(steps), "The number of images must match the number of steps"

    w, h = images[0].size
    font = ImageFont.truetype(font_path, font_size) if font_path else ImageFont.load_default()

    # Calculate the height of the grid including the margin for text
    total_height = h + margin + font_size
    total_width = w * len(images)
    grid_height = total_height + margin + font_size  # Add extra margin for the prompt
    grid = Image.new('RGB', size=(total_width, grid_height), color=(255, 255, 255))
    # Draw the text prompt at the top
    draw = ImageDraw.Draw(grid)
    prompt_text = f"Prompt: \"{prompt}\""
    prompt_width, prompt_height = draw.textbbox((0, 0), prompt_text, font=font)[2:4]
    prompt_x = (total_width - prompt_width) / 2
    prompt_y = margin / 2
    draw.text((prompt_x, prompt_y), prompt_text, fill="black", font=font)

    for i, (image, step) in enumerate(zip(images, steps)):
        # Calculate position to paste the image
        x = i * w
        y = margin + font_size

        # Paste the image
        grid.paste(image, box=(x, y))

        # Draw the step text
        step_text = f"Steps: {step}"
        text_width, text_height = draw.textbbox((0, 0), step_text, font=font)[2:4]
        text_x = x + (w - text_width) / 2
        text_y = y + h + margin / 2 - 8
        draw.text((text_x, text_y), step_text, fill="black", font=font)
    return grid

# Initialize the model pipeline using your local model
model_path = "glyffuser-unconditional/epoch29"  # Path to your trained model
pipeline = DDPMPipeline.from_pretrained(model_path).to("cuda")
pipeline.scheduler = DPMSolverMultistepScheduler()

# Define the number of steps to visualize
num_inference_steps_list = [1, 2, 3, 5, 10, 20, 50]

images = []

# Generate images for each value in num_inference_steps_list
for num_steps in num_inference_steps_list:
    generated_images = pipeline(
        batch_size=1,
        generator=torch.Generator(device='cuda').manual_seed(0),
        num_inference_steps=num_steps
    ).images
    images.append(generated_images[0])  # Append the generated image

# Create the labeled grid with a descriptive prompt since this is an unconditional model
prompt = "Unconditional Diffusion Model Output"
image_grid = make_labeled_grid(images, prompt, num_inference_steps_list)

# Show the grid
from IPython.display import display
display(image_grid)

# Save the image grid
image_grid.save("diffusion_steps_visualization30.png")

### Step 13: Create animated visualization

In [ ]:
from diffusers import DPMSolverMultistepScheduler, DDPMPipeline
from PIL import Image, ImageDraw, ImageFont
import torch
import numpy as np
import os
import imageio.v2 as imageio
from tqdm import tqdm

# Initialize the model pipeline using your local model
model_path = "glyffuser-unconditional/epoch29"  # Path to your trained model
pipeline = DDPMPipeline.from_pretrained(model_path).to("cuda")
pipeline.scheduler = DPMSolverMultistepScheduler()

# Create output directory for frames
os.makedirs("animation_frames", exist_ok=True)

# Set parameters
num_inference_steps = 50
seed = 42

# Use the model's forward process to generate images at each step
print("Generating denoising frames...")

# Start with pure noise (t=1000)
generator = torch.Generator(device="cuda").manual_seed(seed)

# Store all frames
frames = []

# The correct way to visualize the denoising process is to use the pipeline with
# increasing numbers of denoising steps
for step in tqdm(range(0, num_inference_steps + 1, 2)):  # Skip some steps for faster generation
    if step == 0:
        # For the initial noise, just use the pipeline with 1 step
        # This will effectively show the noise
        current_step = 1
    else:
        current_step = step

    # Generate the image with the current number of denoising steps
    image = pipeline(
        batch_size=1,
        generator=torch.Generator(device="cuda").manual_seed(seed),
        num_inference_steps=current_step
    ).images[0]

    # Save the frame
    image.save(os.path.join("animation_frames", f"frame_{step:03d}.png"))
    frames.append(image)

# Create GIF from frames
print("Creating GIF animation...")
# Ensure all frames have the same size (shouldn't be necessary but just in case)
frames_resized = [frame.resize((256, 256)) for frame in frames]

# Save as GIF
output_gif = "diffusion_process30.gif"
frames_resized[0].save(
    output_gif,
    save_all=True,
    append_images=frames_resized[1:],
    optimize=False,
    duration=150,  # milliseconds per frame - slower to see the changes
    loop=0  # 0 means loop indefinitely
)

print(f"Animation saved to {output_gif}")

# Create a grid showing selected frames
def create_process_grid(frames, num_to_show=8):
    # Select frames evenly throughout the process
    if len(frames) <= num_to_show:
        selected_frames = frames
    else:
        indices = np.linspace(0, len(frames)-1, num_to_show, dtype=int)
        selected_frames = [frames[i] for i in indices]

    # Resize frames
    width, height = 256, 256
    selected_frames = [frame.resize((width, height)) for frame in selected_frames]

    # Create grid image
    cols = min(4, num_to_show)
    rows = (num_to_show + cols - 1) // cols

    grid = Image.new('RGB', (width * cols, height * rows))

    for i, frame in enumerate(selected_frames):
        row = i // cols
        col = i % cols
        grid.paste(frame, (col * width, row * height))

    return grid

# Create and save the grid
grid = create_process_grid(frames)
grid.save("diffusion_process_grid3.png")
print("Process grid saved to diffusion_process_grid3.png")